## Day 47 Lecture 2 Assignment

In this assignment, we will perform K-Medoids clustering using responses to a survey about student life at a university.

In [1]:
!pip install pyclustering

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from pyclustering.cluster.kmedoids import kmedoids
import random

This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

Load the dataset.

In [3]:
# answer goes here
students = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv')
print(students.shape)
students.head()

(2958, 36)


,Q2-Participated in Societies and Interest Groups,Q2-Participated in Clubs,"Q2-Participated in Halls, JCRCs and/or Residential College CSCs",Q2-Participated in University organised events,Q3-Interested in Arts & Culture,Q3-Interested in Science & Technology,Q3-Interested in Research and independent study,Q3-Interested in Sports,"Q3-Interested in Other competitions (eg case, debates)",Q3-Interested in Entrepreneurship,Q3-Interested in Volunteering,Q3-Interested in Others,Q4-Passionate about Animal welfare,Q4-Passionate about Arts/Culture/Heritage,Q4-Passionate about Children/Youth,Q4-Passionate about Community building,"Q4-Passionate about Diversity & Inclusion (e.g. special needs, migrant worker, interfaith and intercultural understanding)",Q4-Passionate about Environmental sustainability,Q4-Passionate about Families,Q4-Passionate about Health/Well-being (e.g mental health),Q4-Passionate about Seniors,Q4-Passionate about Poverty reduction,Q4-Passionate about Education,Q4-Passionate about None of the above,Q4-Passionate about Others,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,response_id
0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,2
2,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,3
3,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,1,1,1,1,0,1,0,4
4,1,0,1,1,0,1,1,0,0,1,1,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,1,1,0,1,0,0,0,1,0,5


For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In [4]:
# answer goes here
#only want stress survey questions (begins with Q5)
stress = students.filter(like='Q5', axis=1)
print(stress.shape)
stress.head()

(2958, 10)


,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0
3,1,1,0,1,1,1,1,0,1,0
4,0,1,1,0,1,0,0,0,1,0


The pyclustering implementation of kmedoids supports a variety of distance metrics, but they are primarily for numeric data. We will be using SMC/Hamming dissimilarity and precomputing the similarity matrix (an alternative would be to specify a user-defined function, which you are welcome to try in addition).

We'll assume for the next step that a pair of negative values (i.e. both responses are "no") is as informative as a pair of positive values. Compute the full distance/dissimilarity matrix for the survey data using matching/hamming distance.

In [5]:
# answer goes here
#hamming distance (0-0 = informative)
matrix = pdist(stress, metric='hamming')

#keep as array for better results
hamming = squareform(matrix)
hamming

array([[0. , 0. , 0.1, ..., 0. , 0.2, 0.2],
       [0. , 0. , 0.1, ..., 0. , 0.2, 0.2],
       [0.1, 0.1, 0. , ..., 0.1, 0.1, 0.1],
       ...,
       [0. , 0. , 0.1, ..., 0. , 0.2, 0.2],
       [0.2, 0.2, 0.1, ..., 0.2, 0. , 0.2],
       [0.2, 0.2, 0.1, ..., 0.2, 0.2, 0. ]])

Using the dissimilarity matrix, perform kmedoids clustering using k=2. Set the initial medoids randomly. Note that pyclustering expects the distance matrix to be a numpy array; a pandas dataframe may cause errors. 

Which survey responses are chosen as the cluster representatives? Print out the details of these responses.

In [6]:
# answer goes here
#k=2
num_clusters = 2
initial_medoids = stress.sample(num_clusters).index

kmedoids_instance = kmedoids(hamming, initial_medoids, data_type='distance_matrix')

#fitting
kmedoids_instance.process()

#medoids
medoids = kmedoids_instance.get_medoids()
stress.loc[medoids].style.background_gradient()

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
2,0,1,0,0,0,0,0,1,0,0
1728,0,1,1,0,0,0,0,0,1,0


In [7]:
#clusters
clusters = kmedoids_instance.get_clusters()

stress['cluster'] = 0
for i in range(len(clusters)):
  stress.loc[clusters[i], 'cluster'] = i

stress

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,cluster
0,0,1,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,0
3,1,1,0,1,1,1,1,0,1,0,1
4,0,1,1,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2953,0,1,1,0,0,1,0,0,0,0,1
2954,0,1,0,0,0,0,0,0,0,0,0
2955,0,1,0,0,0,0,0,0,0,0,0
2956,0,1,0,0,0,0,1,1,0,0,0


In [8]:
stress['cluster'].value_counts()

0    1951
1    1007
Name: cluster, dtype: int64

If you run the previous cell a few times, you'll probably notice that the medoids are very sensitive to initialization. A common approach to produce well-separated clusters is to choose initial medoids that are far apart. Re-run the previous process, except with a random pair of medoids that have a dissimilarity of 0.8 or higher. Are the results more stable now? How would you describe the typical clusters you see?

In [9]:
# answer goes here
#turn previous array to df
ham_dist = pd.DataFrame(squareform(matrix), index=stress.index, columns=stress.index)

#use melt
ham_dist['id'] = ham_dist.index
sim_melted = pd.melt(ham_dist, id_vars=['id'], value_vars=ham_dist.columns[ham_dist.columns != 'id'])
sim_melted.columns = ['response_id_1', 'response_id_2', 'dissimilarity']

In [10]:
#look for dissimilarity of 0.8 or >
dissim_high = sim_melted.loc[lambda x: x['dissimilarity'] >= 0.8]
dissim_high

,response_id_1,response_id_2,dissimilarity
245,245,0,0.8
263,263,0,0.8
496,496,0,0.8
752,752,0,0.8
812,812,0,0.8
...,...,...,...
8743789,2899,2955,0.8
8744952,1104,2956,0.8
8745581,1733,2956,0.8
8746809,3,2957,0.8


In [11]:
#finding random pair for analysis
sample = dissim_high.sample(1)
sample 

,response_id_1,response_id_2,dissimilarity
7081937,485,2394,0.8


In [12]:
sample.response_id_1.values[0], sample.response_id_2.values[0]

(485, 2394)

In [13]:
initial_medoids = [sample.response_id_1.values[0], sample.response_id_2.values[0]]

kmedoids_instance = kmedoids(hamming, initial_medoids, data_type='distance_matrix')

#fitting
kmedoids_instance.process()

#medoids
medoids = kmedoids_instance.get_medoids()
stress.loc[medoids].style.background_gradient()

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,cluster
0,0,1,0,0,0,0,0,0,0,0,0
634,1,1,1,0,1,1,0,1,1,0,1


*Are the results more stable now? How would you describe the typical clusters you see?*

I ran the cells a couple of times so my results were different every time. According to this result, it seems that the value counts (shown below) are extremely mismatched compared to the counts when I did not use a sample from dissimilarity >= 0.8. Although the counts are not even the sample version seems to show for the second cluster more answers (more 1's) than in the first cluster. 

In [14]:
#clusters
clusters = kmedoids_instance.get_clusters()

stress['cluster'] = 0
for i in range(len(clusters)):
  stress.loc[clusters[i], 'cluster'] = i

stress

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,cluster
0,0,1,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,0
3,1,1,0,1,1,1,1,0,1,0,1
4,0,1,1,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2953,0,1,1,0,0,1,0,0,0,0,0
2954,0,1,0,0,0,0,0,0,0,0,0
2955,0,1,0,0,0,0,0,0,0,0,0
2956,0,1,0,0,0,0,1,1,0,0,0


In [15]:
stress['cluster'].value_counts()

0    2605
1     353
Name: cluster, dtype: int64